In [48]:
# imports
from PyPDF2 import PdfReader

# local do arquivo
file_path = 'RHPP05LA.PDF'

In [49]:
# correcao de dados
SUBSTITUICOES = {
    "1/12 FERIAS"           : "ferias", 
    "1/12 ferias"           : "ferias", 
    "1/12 DE 1 /3 FERIAS"   : "13_ferias", 
    "1 /3 ferias"           : "13_ferias", 
    "1 /3 FERIAS"           : "13_ferias", 
    "1/12 DE 13_ferias"     : "13_ferias", 
    "TOTAL ferias"          : "total_ferias", 
    "TOTAL FERIAS"          : "total_ferias", 
    "1/12 13 SAL ARIO"      : "13_salario", 
    "13 SAL ARIO"           : "13_salario", 
    "T O T A L"             : "total", 
    "PROVISIONAMENTO"       : "apropriacao", 
    "INSS  20,00%"          : "inss", 
    "SP-PREVCOM"            : "sp-prevcom", 
    "AC TRAB  "             : "ac_trab_", 
    "FGTS 8%"               : "fgts", 
    "TOTAL DE ENCARGOS"     : "total_encargos", 
    "."                     : "", 
    ","                     : ".", 
    "SPPREV"                : "spprev", 
    "FERIAS"                : "ferias"
}

In [50]:
# funcao que divide uma lista em duas listas
# de acordo com a palavra enviada e os pads definidos
def split_tables(
        tb: list, 
        split_word:str="", 
        pad_top_tb1:int=0,
        pad_bottom_tb1:int=0,
        pad_top_tb2:int=0,
        pad_bottom_tb2:int=0
        ):
    
    for i in range(len(tb)):
        if split_word in tb[i]:
            tb1 = tb[pad_top_tb1:i+pad_bottom_tb1]
            tb2 = tb[i+pad_top_tb2:] if pad_bottom_tb2==0 else tb[i+pad_top_tb2:pad_bottom_tb2] 
    return tb1, tb2

# funcao que normatiza palavras-campos das tabelas
def correct_words(tb: list):
    for i in range(len(tb)):
        for search, correct in SUBSTITUICOES.items():
            tb[i] = tb[i].replace(search, correct.lower())
    return tb

# funcao que corrige os dados numericos quebrados por espaco
def data_processing(datas:list):
    for data in datas:
        i = 0
        while i < len(data) - 1:
            if "." not in data[i]:
                data[i] += data[i+1]
                del data[i+1]
            else:
                i += 1
    return datas

# funcao que coloca a tabela em formato de tupla
# formato tabela: 
## linha    0:  cabecalhos
## coluna   0:  tipos
## coluna 1-5:  dados
def cast_table_to_tuple(tb:list) -> tuple:
    headers = tb[0].split()

    datas = tb[1:]
    for i in range(len(datas)):
        datas[i] = datas[i].split()
    
    types = [data[0] for data in datas]

    datas = [data[1:] for data in datas]
    datas = data_processing(datas)
    
    tup = {}
    for type, data in zip(types, datas):
        tup[type] = {}
        for header, value in zip(headers, data):
            tup[type][header] = float(value)

    return tup   

In [51]:
unidades = {}

# ler pdf
with open(file_path, 'rb') as pdf_file:
    pdf_reader = PdfReader(pdf_file)
    pages = pdf_reader.pages

    # para cada page indexada com um number_page
    # extrair texto e colocar em lines (retirando as em branco)
    for number_page, page in enumerate(pages, start=1):
        text = page.extract_text()
        lines = text.split('\n')
        lines = [line for line in lines if not line.isspace()]

        # extrai dados da unidade
        unidade = {}
        unidade['codigo'] = int(lines[3].strip().split(" - ")[0].replace(" ", ""))
        unidade['nome'] = lines[3].strip().split(" - ")[1]

        # se a pagina for impar, os dados sao referentes apropriado e realizado da unidade
        # se a pagina for par, os dados sao referentes aos totais da unidade
        if(number_page %2 == 1):

            # extrai tabela CLT e Autarquico
            tb_clt, tb_aut = split_tables(lines, "A U T A R Q U I C O", 7, -1, 2, -1)

            # extrai apropriado e realizado da tabela
            tb_clt_apropriado, tb_clt_realizado = split_tables(tb_clt, "R E V E R S A O", 1, 0, 1)
            tb_aut_apropriado, tb_aut_realizado = split_tables(tb_aut, "R E V E R S A O", 1, 0, 1)

            # normatiza dados
            tb_clt_apropriado = correct_words(tb_clt_apropriado)
            tb_clt_realizado  = correct_words(tb_clt_realizado)
            tb_aut_apropriado = correct_words(tb_aut_apropriado)
            tb_aut_realizado  = correct_words(tb_aut_realizado)

            #converte as listas para tupla
            unidade['clt'] = {}
            unidade['autarquico'] = {}
            unidade['clt']['apropriado'] = cast_table_to_tuple(tb_clt_apropriado)
            unidade['clt']['realizado']  = cast_table_to_tuple(tb_clt_realizado)
            unidade['autarquico']['apropriado'] = cast_table_to_tuple(tb_aut_apropriado)
            unidade['autarquico']['realizado']  = cast_table_to_tuple(tb_aut_realizado)

            unidades[unidade['codigo']] = unidade

display(unidades) 

{34240: {'codigo': 34240,
  'nome': 'C.RIO CLARO-IB',
  'clt': {'apropriado': {'apropriacao': {'ferias': 121536.56,
     '13_ferias': 40512.2,
     'total_ferias': 162048.76,
     '13_salario': 121536.56,
     'total': 283585.32},
    'inss': {'ferias': 20141.29,
     '13_ferias': 6713.74,
     'total_ferias': 26855.03,
     '13_salario': 20141.29,
     'total': 46996.32},
    'ac_trab_0.5000%': {'ferias': 503.15,
     '13_ferias': 167.71,
     'total_ferias': 670.86,
     '13_salario': 503.15,
     'total': 1174.01},
    'fgts': {'ferias': 8056.28,
     '13_ferias': 2685.46,
     'total_ferias': 10741.74,
     '13_salario': 8056.28,
     'total': 18798.02},
    'sp-prevcom': {'ferias': 0.0,
     '13_ferias': 0.0,
     'total_ferias': 0.0,
     '13_salario': 2244.59,
     'total': 2244.59},
    'total_encargos': {'ferias': 28700.72,
     '13_ferias': 9566.91,
     'total_ferias': 38267.63,
     '13_salario': 30945.31,
     'total': 69212.94},
    'total': {'ferias': 150237.28,
     '13

end
